#### This has code generates Hadamard matrices then produces a widget. This generates all
#### combination of patterns for different numbers of positive samples with various matrix sizes.

### Use the 'cell' in the drop-down menu above then  'run all' to start the calculation. This will produce a   widget window at the end. Use this to produce vector/patterns. The code is written in Python 3 in a Jupyter Notebook running under Anaconda.  Push GO on the widget.

This code is to only be used as a guide. No guarantee is given as to its accuracy. You should use this code only as an aid to writing your own algorithms.

#### Multiplex Detection of Viruses using Hadamard Matrices. G. S. Beddard & B. Yorke, School of Chemistry, University of Leeds, LS2 9JT, UK

https://www.medrxiv.org/content/10.1101/2024.10.21.24315883v2

Email. g.s.beddard@leeds.ac.uk, b.a.yorke@leeds.ac.uk


In [1]:
# import all python 3 add-ons etc that will be needed later on
%matplotlib inline
import numpy as np
import numpy.linalg as La
import matplotlib.pyplot as plt
import time
from itertools import combinations
import ipywidgets.widgets  as wgt
from ipywidgets import interact, interactive, fixed, interact_manual,VBox,HBox,Layout,Output
from IPython.display import display

# Start of Hadamard matrix generation.
### Quadratic residue, Shift Register & Doubling and methods and special Legendre method for $H_{28}$

In [2]:
#------------
def valid_seq_length(n):              # check Hadamard sequence length for Quadratic Residue method only
    maxi = 230
    Hseq = np.zeros(maxi,dtype=int)
    for i in range(maxi):                   # produce Hadamard sequence numbers
        for m in range(0,maxi):
            if isprime(i) and i == 4*m + 3:
                Hseq[i]=i
            pass
    if n in Hseq[0:maxi]:
        is_ok=True
    else:
        is_ok=False
    return is_ok
#------------
# check if integer n is a prime, range starts with 2 and only needs to go up the squareroot of n
def isprime(n):              
    for x in range(2, int(n**0.5)+1):
        if n % x == 0:
            return False
    return True
#------------
# quadratic residue method, this generates more S matrices than the other methods.
def quadratic_hadamard(n):    
    
    init_list = np.zeros(n,dtype=int)

    for i in range(n):                     # make list n/2+1 values = 1 rest zeros so in same ratio as hadamard
        if i <= n//2 : init_list[i] = 1    # check integer division
        pass
    alist = np.zeros(n,dtype=int)
    for i in range(0,(n-1)//2):            # integer division
        alist[(i+1)*(i+1) % n] = 1         # alist = hadamard Srow need only go to half range of n as indices are symmetric
    alist[0] = 1
    Srow = list(alist)
    
    S = np.zeros((n,n),dtype = int)
    for i in range(n):
        for j in range(n):
            S[i,j]= Srow[n-1-j]
        Srow = np.roll(Srow, 1)             # rotate by 1 element at a time
        pass
    
    return S                               # returns S matrix
#------------
#print('Hadamard S matrices by Quadratic residue method')
#for i in range(1,200):
#    if valid_seq_length(i):
#        S = quadratic_hadamard(i)
#        print(i)
#        if i  <= 32 :
#            print('\n'.join( [''.join(['{:2}'.format(item) for item in row] ) for row in S] ) )
#        else:
#            print(''.join(['{:2}'.format(item) for item in S[0]]))
#            xs=''.join( str(S[0][i]) for i in range(len(S[0])) )
#            print(hex(int(xs,2)))
#    pass 

In [3]:
print( 'valid sequence with Quardatic residue method')
for i in range(130):
    if valid_seq_length(i) ==True:
        print(i,',',end='')

valid sequence with Quardatic residue method
0 ,3 ,7 ,11 ,19 ,23 ,31 ,43 ,47 ,59 ,67 ,71 ,79 ,83 ,103 ,107 ,127 ,

In [4]:
#-------------
# shift register method S matrix size  2^k-1 , k = 2, 3, 4  etc
def shift_hadamard(num):                
    num = int(num) + 1
    def S_lineA(n,m):
        SL     = np.zeros(n,dtype=int)
        SL[0] = 1                               # set x^n = 1
        for j in range(2**n-1):

            tmp = (SL[m] + SL[0]) % 2           # mod 2
            SL = np.roll(SL,-1)                 # shift array elements
            SL[n-1] = tmp                       # last one as x^n
            Srow[j] = SL[0]
        pass

    def S_lineB(n,ma,mb,mc):
        SL    = np.zeros(n,dtype=int)
        SL[0] = 1                               # set x^n = 1
        for j in range(2**n-1):
            tmp1 = (SL[ma] + SL[0]) % 2         # mod 2
            tmp2 = (SL[mb] + tmp1 ) % 2
            tmp  = (SL[mc] + tmp2 ) % 2
            SL = np.roll(SL,-1)
            SL[n-1] = tmp                       # last one as x^n
            Srow[j] = SL[0]
        pass

    for k in range(num-1,num):                  # can generate v large matrices, 2^8-1, 2^9-1
        n = 2**k-1
        Srow = np.zeros(n,dtype=int)            # holds one row of S matrix
        if k in [2,3,4,6,7,15]:	S_lineA(k,1)
        if k in [5,11]:	S_lineA(k,2)
        if k == 8:      S_lineB(k,1,5,6)
        if k == 9:      S_lineA(k,4)
        if k == 10:     S_lineA(k,3)
        if k == 12:     S_lineB(k,3,4,7)
        if k == 13:     S_lineB(k,1,3,4)
        if k == 14:     S_lineB(k,1,11,12)
        pass

        S = np.zeros((n,n),dtype = int)
        for i in range(n):
            for j in range(n):
                S[i,j] = Srow[n-1-j]
            Srow = np.roll(Srow, 1)                   # rotate by 1 element at a time
            pass

    return S
#------------
# 2^k+1  is size of matrix side
S = shift_hadamard(4)
print(S)

[[1 1 1 1 0 1 0 1 1 0 0 1 0 0 0]
 [1 1 1 0 1 0 1 1 0 0 1 0 0 0 1]
 [1 1 0 1 0 1 1 0 0 1 0 0 0 1 1]
 [1 0 1 0 1 1 0 0 1 0 0 0 1 1 1]
 [0 1 0 1 1 0 0 1 0 0 0 1 1 1 1]
 [1 0 1 1 0 0 1 0 0 0 1 1 1 1 0]
 [0 1 1 0 0 1 0 0 0 1 1 1 1 0 1]
 [1 1 0 0 1 0 0 0 1 1 1 1 0 1 0]
 [1 0 0 1 0 0 0 1 1 1 1 0 1 0 1]
 [0 0 1 0 0 0 1 1 1 1 0 1 0 1 1]
 [0 1 0 0 0 1 1 1 1 0 1 0 1 1 0]
 [1 0 0 0 1 1 1 1 0 1 0 1 1 0 0]
 [0 0 0 1 1 1 1 0 1 0 1 1 0 0 1]
 [0 0 1 1 1 1 0 1 0 1 1 0 0 1 0]
 [0 1 1 1 1 0 1 0 1 1 0 0 1 0 0]]


In [5]:
#--------------
# S matrix size (2^k) -1 , k = 2, 3, 4 etc, replace each element by previous matrix
def doubling_hadamard(max_size):    
    
    print('Matrix doubling method size (2^k)-1, k=2,3.. :  k=',max_size , 'S matrix is not circulant')
    max_size = int(max_size)
    h0 = np.ones( (2,2),dtype = int )     # define initial Hadamard matrix
    h0[1,0] = -1
    #h0[1,1]=-1    # this way round leading row & col are 1's 
    print('initial form\n',h0,h0 @ h0.T)
    n = 2  # must be 2
    Htemp = h0
    for i in range(1,max_size):
        print(' {:s} {:d}'.format(' calculation continues, S size = ',2*2**i-1))
        Hnn = np.zeros((2*n,2*n),dtype=int)
        h0, h1 = Htemp.shape
        Hnn[0  : 0  + h0   , 0  : 0  + h1] =  Htemp
        Hnn[n  : n  + h0   , 0  : 0  + h1] = -Htemp 
        Hnn[0  : 0  + h0   , n  : n  + h1] =  Htemp
        Hnn[n  : n  + h0   , n  : n  + h1] =  Htemp
        print('Hnn\n',Hnn)
        n = 2*n
        s = Hnn.shape
        Htemp = np.zeros( s ,dtype = int)
        Htemp = Hnn[ 0:s[0],  0:s[0] ]

        sm = s[0] - 1
        Tmpmat = np.zeros((sm,sm),dtype=int)
        Smat   = np.zeros((sm,sm),dtype=int)
        Tmpmat= Htemp[1:sm+1, 0:sm]
        for ii in range(sm):
            for j in range(sm):
                if Tmpmat[ii,j] == 1:
                    Smat[ii,j]= 0
                else:
                    Smat[ii,j]= 1
                pass
    print('-------------------------------')
    return Smat
#-------------

smat = doubling_hadamard(4)                   # 4 means  15 x 15 matrix
print('final S matrix \n',smat.shape)
print('\n'.join( [''.join(['{:2}'.format(item) for item in row] ) for row in smat] ) )

Matrix doubling method size (2^k)-1, k=2,3.. :  k= 4 S matrix is not circulant
initial form
 [[ 1  1]
 [-1  1]] [[2 0]
 [0 2]]
  calculation continues, S size =  3
Hnn
 [[ 1  1  1  1]
 [-1  1 -1  1]
 [-1 -1  1  1]
 [ 1 -1 -1  1]]
  calculation continues, S size =  7
Hnn
 [[ 1  1  1  1  1  1  1  1]
 [-1  1 -1  1 -1  1 -1  1]
 [-1 -1  1  1 -1 -1  1  1]
 [ 1 -1 -1  1  1 -1 -1  1]
 [-1 -1 -1 -1  1  1  1  1]
 [ 1 -1  1 -1 -1  1 -1  1]
 [ 1  1 -1 -1 -1 -1  1  1]
 [-1  1  1 -1  1 -1 -1  1]]
  calculation continues, S size =  15
Hnn
 [[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1]
 [-1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1]
 [-1 -1  1  1 -1 -1  1  1 -1 -1  1  1 -1 -1  1  1]
 [ 1 -1 -1  1  1 -1 -1  1  1 -1 -1  1  1 -1 -1  1]
 [-1 -1 -1 -1  1  1  1  1 -1 -1 -1 -1  1  1  1  1]
 [ 1 -1  1 -1 -1  1 -1  1  1 -1  1 -1 -1  1 -1  1]
 [ 1  1 -1 -1 -1 -1  1  1  1  1 -1 -1 -1 -1  1  1]
 [-1  1  1 -1  1 -1 -1  1 -1  1  1 -1  1 -1 -1  1]
 [-1 -1 -1 -1 -1 -1 -1 -1  1  1  1  1  1  1  1  1]
 [ 1 -1  

In [6]:
#This Python code was originally based on the R code of 
#J. Steepleton, "Constructions of Hadamard Matrices" (2019). 
#Chancellor’s Honors Program Projects. https://trace.tennessee.edu/utk_chanhonoproj/2266

# matrix size;  m = 28  so divisible by 4 
# 2^k(p+1) = 28 when k=2, p=13

#-------------------------------
def legendre(p):                      # Legendre symbols, see Wikipedia
    leg = np.zeros(p,dtype=int)
    for i in range(p):
        temp = i**((p-1)/2) % p
        if temp > 1:
            temp = -1
        leg[i] = temp
    return leg[1:]                    # remove first
#--------------------------------

def Hadamard28(L,p):                       # form Hadamard 28 by 28 matrix
    
    def getValues(i,j,a,b,c,d):
        n = 2*(p+1)
        if 2*i-1 < 0 and 2*j-1 < 0:
            Had28[n-1,n-1]      = a
        else:
            Had28[i+i-1  ,j+j-1]= a
        
        if 2*i-1 < 0 and 2*j >= 0:
            Had28[n-1,j+j]      = b
        else:
            Had28[i+i-1,j+j]    = b 
            
        if 2*i >= 0 and 2*j-1 < 0:
            Had28[i+i,n-1]     = c
        else:
            Had28[i+i  ,j+j-1] = c
        
        Had28[i+i    ,j+j]   = d
    pass
        
    #----------------------------------- 
    
    Had28 = np.zeros((2*(p+1),2*(p+1)),dtype=int)
    B     = np.zeros((p+1,p+1),dtype=int)      # insert Q into B as 2nd row & second column
    Q     = np.zeros((p,p),dtype=int)
    Q[0,1:] = L[:]                         # Legendre
    #print('***',Q)
    for i in range(1,p):                   # roll vector all except first 
        Q[i,:] = np.roll(Q[0,:],i)
    Q[0,0] = 0                             # from now on is book-keeping, 
                                           # insert Q into B as 2nd row & second column  
    for i in range(p+1):
        B[0,i] = 1
        B[i,0] = 1
    for i in range(1,p+1):
        for j in range(1,p+1):
            B[i,j] = Q[i-1, j-1]
    B[0,0] = 0
    for i in range(0,p+1):                  # S matrix so fill matrix with 1& 0 instead of 1, -1
        for j in range(0,p+1):              # index [-1] is last element in array
            
            if B[i,j] == 1: 
                getValues(i,j,1,1,1,0)
                
            if B[i,j] == -1:
                getValues(i,j,0,0,0,1)

            if B[i,j] == 0: 
                getValues(i,j,1,0,0,0)
            pass
    return Had28 
#----------------------------------------
# this runs code safely  

if __name__== '__main__':

    p = 13                                        # size = 2^k(p+1), in this case k=1
    L     = legendre(p)
    Had28 = Hadamard28(L,p)

    S28 = Had28[:-1,:-1]                          # make S matrix
    
    print('print S = 27 matrix,\nsize = ',len(S28[0,:]),len(S28[:,0])  ) 
    pass

for i in range(0,2*(p+1)-1):
    print('{:4d}'.format(i),end='  |  ')
    for j in range(0,2*(p+1)-1):
        temp = S28[j,i]
        print(temp,end=' ')
    print()
print()


print S = 27 matrix,
size =  27 27
   0  |  0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 
   1  |  1 1 0 1 1 0 0 1 1 1 1 0 0 0 0 0 0 0 0 1 1 1 1 0 0 1 1 
   2  |  0 0 0 1 0 0 1 1 0 1 0 0 1 0 1 0 1 0 1 1 0 1 0 0 1 1 0 
   3  |  1 1 1 1 0 1 1 0 0 1 1 1 1 0 0 0 0 0 0 0 0 1 1 1 1 0 0 
   4  |  0 1 0 0 0 1 0 0 1 1 0 1 0 0 1 0 1 0 1 0 1 1 0 1 0 0 1 
   5  |  1 0 0 1 1 1 0 1 1 0 0 1 1 1 1 0 0 0 0 0 0 0 0 1 1 1 1 
   6  |  0 0 1 1 0 0 0 1 0 0 1 1 0 1 0 0 1 0 1 0 1 0 1 1 0 1 0 
   7  |  1 1 1 0 0 1 1 1 0 1 1 0 0 1 1 1 1 0 0 0 0 0 0 0 0 1 1 
   8  |  0 1 0 0 1 1 0 0 0 1 0 0 1 1 0 1 0 0 1 0 1 0 1 0 1 1 0 
   9  |  1 1 1 1 1 0 0 1 1 1 0 1 1 0 0 1 1 1 1 0 0 0 0 0 0 0 0 
  10  |  0 1 0 1 0 0 1 1 0 0 0 1 0 0 1 1 0 1 0 0 1 0 1 0 1 0 1 
  11  |  1 0 0 1 1 1 1 0 0 1 1 1 0 1 1 0 0 1 1 1 1 0 0 0 0 0 0 
  12  |  0 0 1 1 0 1 0 0 1 1 0 0 0 1 0 0 1 1 0 1 0 0 1 0 1 0 1 
  13  |  1 0 0 0 0 1 1 1 1 0 0 1 1 1 0 1 1 0 0 1 1 1 1 0 0 0 0 
  14  |  0 0 1 0 1 1 0 1 0 0 1 1 0 0 0 1 0 0 1 1 0 1 0 0 1 0 1 
  15 

# End of Hadamard matrix generation¶

# Start of calculation to produce patterns for different size matrices and number of positives

In [7]:

def make_rows(m):                # type 2 matrix
    if m == 7:  
        letters = ['A','D','F','G']
    if m == 11:
        letters = ['A', 'C', 'G', 'H', 'I', 'K']                        #  as in table in paper
    if m == 15:
        #letters = ['A', 'B', 'C', 'D', 'E', 'F','G','H','M','N']       # best on 3 and 4 groups
        letters = ['A', 'B', 'C', 'D', 'E', 'F', 'I', 'J', 'K', 'O']    # 4 with 4 + 6 with 6.
        #letters = ['A', 'B', 'C', 'D', 'E', 'F', 'H', 'J', 'K', 'N']   # doubling method 4 x 6 + 4*4
    if m == 19:
        #letters = ['A', 'D', 'G', 'I', 'K', 'M','N','O','P','S'] 
        letters = ['A', 'C', 'D', 'I', 'K', 'M', 'N', 'O', 'P', 'S']    # all fives
    if m == 23:
        letters = ['A','F','H','I','K','O','P','R','T','V','W']
    if m == 27:
        letters = ['A','B','C','D','E','F','G','H','J','M','O','T','X']
    if m == 31:
        letters = ['C','D','G','J','L','N','P','U','V','X','Y','a','e']
    rows    = np.zeros(len(letters),dtype=int)
    for i in range(len(rows) ):
        if ord(letters[i]) < 91:                                 # ascii A = 65,  Z = 90
            rows[i] = ord(letters[i]) - 65                       # is capital A = 1, Z = 26
        else:
            rows[i] = ord(letters[i]) - 97 + 27 -1               # ascii 97 is lower case a
    return rows
#---------------------------------
def make_rows_type1(m):                # type 1 matrix 
    if m == 7:  
        letters = ['A','D','F']
    if m == 11:
        letters = ['B', 'D', 'E', 'G']                     
    if m == 15:
        letters = ['B', 'C', 'D', 'E']    
    if m == 19:   
        letters = ['B', 'E', 'F', 'H', 'N']    
    if m == 23:
        letters = ['A','H','K','N','Q']
    if m == 27:
        letters = ['A','B','E','N','V']
    if m == 31:
        letters = ['B','C','E','G','L','O']
    rows    = np.zeros(len(letters),dtype=int)
    for i in range(len(rows) ):
        if ord(letters[i]) < 91:                                 # ascii A = 65,  Z = 90
            rows[i] = ord(letters[i]) - 65                       # is capital A => 1, Z => 26
        else:
            rows[i] = ord(letters[i]) - 97 + 27 - 1               # ascii 97 is lower case a
    return rows
#---------------------------------

def make_reduced_matrix(m,rows,S):                       # Reduced S matrix, remove last column
    Sred = np.zeros((len(rows),m-1), dtype =int)
    for k,val in enumerate(rows):
        arry = S[val, 0:m - 1]
        Sred[k, 0:m - 1] = arry
        pass
    return Sred
#--------------------------

def do_row_sums(data,Sred):                   # Calculate down columns to get sum
    row, col = Sred.shape
    #print('row col', row,col)
    dotp = np.zeros(row, dtype=float)
    for i in range(row):                      # dot product. Make vector dotp
        dotp[i] = np.dot(Sred[i,:] , data[:])
    return dotp
#-----------------------------

def test_for_positives(data,Sred,pnum):
    
    dotpvec = do_row_sums(data,Sred)
    if sum(dotpvec) == 0:
        temp = np.zeros(len(dotpvec),dtype=int)# 0 positive
        indx = [-10 for i in range(10)]
        indx[0] = -2
        flag = 0
        print(temp,' Zero pos')
        pnum[0] = pnum[0] + 1
    else:                                               # samples present 
        
        pnum, q,   indx, flag,total_column,total_numbs,total_ones = one_sample(Sred,   dotpvec,comb_indx1,pnum,total_column,total_numbs,total_ones)
        if q == 0:
            pnum, q,indx,flag,total_column,total_numbs,total_ones = two_samples(Sred,  dotpvec,comb_indx2,pnum,total_column,total_numbs,total_ones)
        if q == 0:
            pnum, q,indx,flag,total_column,total_numbs,total_ones = three_samples(Sred,dotpvec,comb_indx3,pnum,total_column,total_numbs,total_ones)
        if q == 0:
            print('more than 3,  vector =',dotpvec)
            indx = [-10 for i in range(10)]
            indx[0] = -1
            pnum[7] = pnum[7] + 1
            flag = 1
    return pnum ,indx,flag                                            # number of positives in this array
#--------------------------------------
def allcombs(alist, num):
    return list(combinations(alist,num))
#---------------------------------------


# WIDGET for checking pattern in type 2 matrices

In [8]:
def find_one_sample(m,dotpvec,Sred):
    
    q = 0
    indx = [-10]
    comb_indx = allcombs([i for i in range(m-1)], 1)
    Lindx=len(comb_indx)
    temp = np.zeros(len(dotpvec),dtype=int)
    for i in range(Lindx):
        w = comb_indx[i]
        a0 = w[0]                               # a0  is index for Sred array
        temp[:] = Sred[:,a0]                    #  column a0
        if np.array_equal(temp, dotpvec):
            #print('\nOne positive at position ', a0+1 )
            indx[0] = a0 + 1
            q = q + 1
            pass
        pass
    return q, indx[:]
#--------------------------------------
def find_two_samples(m,dotpvec,Sred):
    q = 0
    indx = [-10,-10]
    comb_indx = allcombs([i for i in range(m-1)], 2)
    temp = np.zeros(len(dotpvec),dtype=int)
    for i in range(len(comb_indx)):
        w = comb_indx[i]
        a0 = w[0]                               # a0,a1 is index for Sred array
        a1 = w[1]
        temp[:] = Sred[:,a0] + Sred[:, a1]      # two columns a0, and a1
        if np.array_equal(temp, dotpvec):
            #print('\nTwo positives at positions ', a0+1,' and ', a1+1 )
            q = q + 1
            indx[:] = [a0 + 1, a1 + 1 ]
            pass
        pass
    return  q, indx[:]
#-------------------------------------
def find_three_samples(m,dotpvec,Sred):
    q = 0
    indx = [-10,-10,-10]
    comb_indx = allcombs([i for i in range(m-1)], 3)
    temp = np.zeros(len(dotpvec),dtype=int)
    for i in range(len(comb_indx)):
        w = comb_indx[i]
        a0 = w[0]                               # a0,a1 is index for Sred array
        a1 = w[1]
        a2 = w[2]
        temp[:] = Sred[:,a0] + Sred[:, a1] +Sred[:,a2]      # two columns a0, and a1
        if np.array_equal(temp, dotpvec):
            #print('\n3 positives at positions ', a0+1, a1+1, a2+1 )
            q = q + 1
            indx[:] = [a0 + 1, a1 + 1 , a2 + 1]
            pass
        pass
    return  q, indx[:]
#-------------------------------------
def find_four_samples(m,dotpvec,Sred):
    q = 0
    indx = [-10,-10,-10,-10]
    comb_indx = allcombs([i for i in range(m-1)], 4)
    temp = np.zeros(len(dotpvec),dtype=int)
    for i in range(len(comb_indx)):
        w = comb_indx[i]
        a0 = w[0]                               # a0,a1 is index for Sred array
        a1 = w[1]
        a2 = w[2]
        a3 = w[3]
        temp[:] = Sred[:,a0] + Sred[:, a1] +Sred[:,a2] +Sred[:,a3]      # two columns a0, and a1
        if np.array_equal(temp, dotpvec):
            #print('\n3 positives at positions ', a0+1,' and ', a1+1, a2+1, )
            q = q + 1
            indx[:] = [a0 + 1, a1 + 1, a2 + 1, a3 + 1]
            pass
        pass
    return  q, indx[:]
#-------------------------------------

def pattern_test(m,typ,pattern):          # type 2 matrix paterns only
    
    indx = []
    q = 0
    lenp = len(pattern)
    sump = np.sum(pattern)                 # number of 1's in each column
    if sump == 0:
        q = -2
        return q, indx[:]                  # no positive found so quit
    
    if typ == 2:
        rows = make_rows(m)                # this def is defined above
    else:
        rows = make_rows_type1(m)
    
    if m in [7, 11, 19, 23, 31]:
        S = quadratic_hadamard(m)          # this def is defined above     
    
    if m == 15:
        S = shift_hadamard(4)              # this def is defined above
        
    if m == 27:
        p = 13                             # size = 2^k(p+1), in this case k=1
        L = legendre(p)
        S = Hadamard28(L,p)
     
    if typ == 1:
        mm = m + 1
    else:
        mm = m
    Sred = make_reduced_matrix(mm,rows,S)   # this def is defined above
    
                                           # now start with each case
    q = -1
    if m == 7 and typ == 2:                             # m = 7
        if sump == 2:                                 # sump is sum of columns
            q,indx = find_one_sample(m,pattern,Sred) 
        elif sump == 4:
            q,indx = find_two_samples(m,pattern,Sred)
        elif sump == 6:
            q,indx = find_three_samples(m,pattern,Sred)
        elif sump == 8:
            q,indx = find_four_samples(m,pattern,Sred)
    if m == 7 and typ == 1:
        if sump in [1,2,3]:
            q,indx = find_one_sample(mm,pattern,Sred) 
            
    if m == 11 and typ == 2:                             # m = 11
        if sump == 3:
            q,indx = find_one_sample(m,pattern,Sred)
        elif sump == 6:
            q,indx = find_two_samples(m,pattern,Sred) 
        elif sump == 9:
            q,indx = find_three_samples(m,pattern,Sred) 
    if m == 11 and typ == 1:
        if sump in [1,2,3,4]:
            q,indx = find_one_sample(mm,pattern,Sred)
    
    if m == 15 and typ == 2:                             # m = 15 
        if sump in [4,6]:
            q,indx = find_one_sample(m,pattern,Sred) 
        if sump in [8,10,12]:
            q,indx = find_two_samples(m,pattern,Sred)
        if sump in [12,14,16,18]:
            q,indx = find_three_samples(m,pattern,Sred)    
    if m == 15 and typ == 1:
        if sump in [1,2,3,4]:
            q,indx = find_one_sample(mm,pattern,Sred)
    
    
    if m == 19 and typ == 2:                             # m = 19
        if sump == 5:
            q,indx = find_one_sample(m,pattern,Sred) 
        elif sump == 10:
            q,indx = find_two_samples(m,pattern,Sred)      
    if m == 19 and typ == 1:
        if sump in [1,2,3,4,5]:
            q,indx = find_one_sample(mm,pattern,Sred)
    
    
    if m == 23 and typ == 2:                             # m = 23
        if sump in [4,5,6,7]:
            q,indx = find_one_sample(m,pattern,Sred) 
        elif sump in [8,9,10,11,12,13,14]:
            q,indx = find_two_samples(m,pattern,Sred)         
    if m == 23 and typ == 1:
        if sump in [1,2,3,4,5]:
            q,indx = find_one_sample(mm,pattern,Sred)
            
    if m == 27 and typ == 2:                             # m = 27
        if sump in [3,4,5,6,7]:
            q,indx = find_one_sample(m,pattern,Sred) 
        elif sump in [7,8,9,10,11,12,13,14,15,16,17]:
            q,indx = find_two_samples(m,pattern,Sred) 
            
    if m == 27 and typ == 1:
        if sump in [1,2,3,4,5]:
            q,indx = find_one_sample(mm,pattern,Sred)
      
    return q, indx[:]                                                                                                                                                                                                                                                                              
#-------------------------------

def getlist(m, atyp, astr):
    
    xastr=''.join( i for i in astr if i in  '0123456789') #  numeric only
    lens  = len(xastr)
    xlist = [-1 for i in range(lens) ]
    for i in range(lens):
        xlist[i]=  ord(xastr[i]) - 48
        if xlist[i] not in [0,1,2,3,4,5,6,7,8,9]:  # check that all are numbers
            xx = 0
            return '** Fault, letters are not allowed', xlist
            break
                                                # pre check that list is ok
    
    mlen      = [ 7, 11, 15, 19, 23, 27, 31]    # S matrix size
    Smat_typ1 = [ 3,  4,  4,  5,  5,  5,  6]    # number of rows type 1
    Smat_typ2 = [ 4,  6, 10, 10, 11, 13, 13]    # number of rows type 2

    xx = 0
    if atyp == 2:
        for i in range(len(mlen)):
            if m == mlen[i] and len(xlist) == Smat_typ2[i]:
                xx = 1
                break
    if atyp == 1:
        for i in range(len(mlen)):
            if m == mlen[i] and len(xlist) == Smat_typ1[i]:
                xx = 1
                break
    if xx == 0:
        outstr= '** Fault in number of digits for size of matrix'
        indx = []
        
    else:
        q, indx = pattern_test(m,atyp,xlist)
        if q == -2:
            outstr = 'No positive sample present'
        if q == -1:
            outstr = '** Fault, sum digits / pattern wrong'
        if q == 0:
            outstr = '** Fault in pattern, no match found'
        if q > 1:
            outstr = '** Fault, ambiguous result re-check needed ' 
        if q == 1 and len(indx) == 1:
            outstr = 'Positive at '+ str(indx[0])
        if q == 1 and len(indx) == 2:
            outstr = 'Positives at '+ str(indx[0]) + ' and ' + str(indx[1])
        if q == 1 and len(indx) == 3:
            outstr = 'Positives at '+ str(indx[0]) + ', ' + str(indx[1])+ ' and ' + str(indx[2])
        if q == 1 and len(indx) == 4:
            outstr = 'Positives at '+ str(indx[0]) + ', ' + str(indx[1])+ ', ' + str(indx[2])+' and '+str(indx[3])
    
    return outstr,xlist
#----------------------------------

In [9]:
# Widget for checking pattern in type 2 matrices
# only goes as far a 3 positives.

def Reset_button_clicked(b):        # reset
    
    text1.value = ''
    text2.value = ''
    text3.value = ''
    pass
#---------------------------------------
def clr_button_clicked(b):
    outpt.clear_output()
#---------------------------------
def Go_button_clicked(b):                      # read values and do calculation
    
    atyp = int(radio_type.value)               # Read value from radio button. Make into integer           
    m    = int(radio_matrix.value)
    astr = str(text1.value)
    
    bstr, xlist  = getlist(m, atyp, astr )     # checking done here
    
    text3.value = bstr
    text2.value = str(xlist).replace('[','').replace(']','')  # put result into box on screen
    
    with outpt:
        print('{:s} {:d} {:s} {:2d} {:s}   {:s}'.format('matrix type =', atyp, 'size =', m, bstr, str(xlist) ) )

    pass
#----------------------------------
def make_boxes():
    vbox1 = wgt.VBox([wgt.Label('Analysis with Hadamard reduced S matrices'),L0,L00, text1, b2, b3,bclr, Lbox])
    vbox2 = wgt.VBox([wgt.Label('Choose matrix type and size'), radio_type, radio_matrix])
    return vbox1, vbox2
#----------------------------------
L0 = wgt.Label('Input pattern, e.g. 0,0,0,1,1,1 then click GO! ' )
L00 = wgt.Label('No more than 3 positives are checked' )

radio_type    = wgt.RadioButtons(value='2' , options=['1', '2'], description='Matrix type')
radio_matrix  = wgt.RadioButtons(value='11', options=['7', '11', '15', '19', '23','27'], description='S matrix size')

text1 = wgt.Text(value='1, 0, 2, 1, 1, 1',disabled=False,width='20%')

text2 = wgt.Text(value='', disabled = False, width ='20%')

Lbox  = VBox([wgt.Label('Pattern read as'), text2])

b2    = wgt.Button(description='GO',    style=dict(button_color='lightgreen',font_weight='bold'))
b2.on_click(Go_button_clicked)

b3    = wgt.Button(description='Reset', style=dict(button_color='lightblue',font_weight='bold') )
b3.on_click(Reset_button_clicked)

bclr    = wgt.Button(description='Clear output',style = dict(button_color='orange',font_weight='bold'))
bclr.on_click(clr_button_clicked)

text3 = wgt.Text(value='',disabled=False,width='20%')
result= VBox([wgt.Label('Position(s) of any positives'), text3])

outpt = wgt.Output(layout={'border': '3px solid black'})

vbox1, vbox2 = make_boxes()
display(wgt.HBox([vbox1, vbox2]),result,outpt)


Output(layout=Layout(border='3px solid black'))


Example patterns Patterns  use numbers with spaces or with commas etc. 

$$\begin{array}{lll}
[1 0 0 2 2 1] & n = 11   & \text{sites 3 and 10}\\
[0  0, 0, 1, 1, 1, 1, 0, 1, 1])   & n=15 & \text{site 13}\\
[0, 0, 1, 1, 1, 1, 2, 1, 1, 2)   &n=15   & \text{sites 10 and 13 }\\
[0, 0, 1, 0, 1, 2, 1, 1, 1, 2]   &n=15   & \text{error in pattern, does not exist}\\
[0, 0, 1, 0, 1, 1, 2, 1, 1, 2]   &n=15   & \text{error in pattern + does not exist sum digits wrong}\\
[2, 1, 0, 1, 2, 1, 2, 1, 0, 0]   &n= 19  &  \text{sites 2 and 8 . first is zero}\\
[0 2 1 1 0 2 1 2 0 1 2]         & n= 23  &  \text{sites 6 & 16}\\
[2, 1, 0, 1, 2, 1, 2, 1, 0, 0,] & n = 19 & \text{sites 2 and 8}\\
\end{array}$$
